# iPython notebook to parse Tableau workbook for users
## Importing libraries

In [ ]:
import os, argparse, keyring, re, configparser, warnings, urllib, requests
from datetime import time, datetime
from getpass import getpass
warnings.filterwarnings('ignore')

## Functions
### Fn to sign in to Server

In [ ]:
def sign_in(username, password, site=""):
    body = {
        "credentials": {
            "name": username,
            "password": password,
            "site": {
                "contentUrl": site
            }
        }
    }
    credentials = requests.post(
        URL + '/auth/signin', 
        json=body, 
        verify=False, 
        headers={'Accept': 'application/json'}
    ).json()['credentials']
    return credentials['site']['id'], credentials['token']

### Fn to sign out from Server

In [ ]:
def sign_out(site_id, token):
    requests.post(
        URL + '/auth/signout', 
        verify=False, 
        headers={'Accept': 'application/json',
                'X-Tableau-Auth': token}
    )

### Fn to find workbooks

In [ ]:
def find_workbooks(site_id, token, workbook_name):
    response = requests.get(
        URL + '/sites/{}/workbooks?filter=name:eq:{}'.format(site_id, workbook_name), 
        verify=False, 
        headers={
            'Accept': 'application/json',
            'X-Tableau-Auth': token
        }
    ).json()
    return [workbook['id'] for workbook in response['workbooks']['workbook']]

### Fn to find views in the workbook

In [ ]:
def find_views(site_id, token, workbook_id, view_name):
    response = requests.get(
        URL + '/sites/{}/workbooks/{}'.format(site_id, workbook_id), 
        verify=False, 
        headers={
            'Accept': 'application/json',
            'X-Tableau-Auth': token
        }
    ).json()
    for view in response['workbook']['views']['view']:
        if view['name'] == view_name:
            return view['id']
    return 0

### Fn to find the users on the Server with the username

In [ ]:
def query_user(site_id, token, username):
    query_response = requests.get(
        URL + '/sites/{}/users?filter=name:eq:{}'.format(site_id, username), 
        verify=False, 
        headers={
            'Accept': 'application/json',
            'X-Tableau-Auth': token
        }
    ).json()
    user_id = query_response['users']['user'][0]['id']
    user_role = query_response['users']['user'][0]['siteRole']
    
    response = requests.get(
        URL + '/sites/{}/users/{}'.format(site_id, user_id), 
        verify=False, 
        headers={
            'Accept': 'application/json',
            'X-Tableau-Auth': token
        }
    ).json()
    
    user_details = "{} ({}) | {}".format(response['user']['fullName'], response['user']['name'], response['user']['siteRole'])
    return user_details, user_role

### Fn to unlicense users on the Server

In [ ]:
def unlicense_users(site_id, token, username):
    query_response = requests.get(
        URL + '/sites/{}/users?filter=name:eq:{}'.format(site_id, username), 
        verify=False, 
        headers={
            'Accept': 'application/json',
            'X-Tableau-Auth': token
        }
    ).json()
    user_id = query_response['users']['user'][0]['id']
    
    unlicense_body = {
        "user": {
            "siteRole": "Unlicensed"
        }
    }
    
    response = requests.put(
        URL + '/sites/{}/users/{}'.format(site_id, user_id), 
        json=unlicense_body, 
        verify=False, 
        headers={
            'Accept': 'application/json',
            'X-Tableau-Auth': token
        }
    ).json()
    return response

### Fn to read the data from a dashboard

In [ ]:
def query_data(site_id, token, view_id):
    response = requests.get(
        URL + '/sites/{}/views/{}/data'.format(site_id, view_id), 
        verify=False, 
        headers={
            'Accept': 'application/json',
            'X-Tableau-Auth': token
        }
    ).text.splitlines()
    response.remove(response[0])
    return response

## Variables

In [ ]:
server = "penguin.biztory.com"
site_content_url = "PraveenSam"
api_ver = "3.8"
username = "praveen.sam@biztory.be"

URL = "https://{}/api/{}".format(server, api_ver)
workbook_name = "UnusedUsers"
view_name = "Users not signed in last one month"
workbook_name = workbook_name.replace(" ", "+")

## Logging in to the Tableau Server

In [ ]:
password = getpass("Enter your password for the Tableau Server: ")
site_id, token = sign_in(username, password, site_content_url)

## Open Log file

In [ ]:
log_file = open("user_remove.log", "a+")

## Query users from dashboard

In [ ]:
view_id = find_views(site_id, token, find_workbooks(site_id, token, workbook_name)[0], view_name)
users_to_remove = query_data(site_id, token, view_id)

In [ ]:
print(users_to_remove)

## Write data to log

In [ ]:
dt_string = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
num_users_removed = len(users_to_remove)

log_file.write("\n\n{}: User removal script run - {} users removed from the {} Site on {}".format(dt_string, num_users_removed, site_content_url, server))

## Unlicense users

In [ ]:
for user in users_to_remove:
    user_detail, user_role = query_user(site_id, token, user)
    if user_role == "ServerAdministrator":
        print("\nCannot unlicense Server Administrator!! Skipped {}!".format(user_detail))
        log_file.write("\nCannot unlicense Server Administrator!! Skipped {}!".format(user_detail))
    else:
        print("\nUnlicensed {} from {} site".format(user_detail, site_content_url))
        log_file.write("\nUnlicensed {}...".format(user_detail))
        unlicense_users(site_id, token, user)

## Signing out of the Tableau Server

In [ ]:
print("\n\tSigning out of " + server)
sign_out(site_id,token)